In [23]:
import pandas as pd
import numpy as np
import os

### Read in the data

In [24]:
df = pd.read_csv('https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/predictor/player_offense.csv')

In [25]:
#convert column types to appropriate data types
df = df.astype({'seas': 'category',
          'wk': 'category',
          'team': 'category',
          'pos1': 'category',
          'dv': 'category',
          'seas.1': 'category'})

In [26]:
#convert dob to datetime
df['dob'] = pd.to_datetime(df['dob'])

In [28]:
#rename seas.1 columns
df.rename(columns={'seas.1': 'exp'}, inplace=True)

Index(['pk', 'gid', 'seas', 'wk', 'player', 'team', 'pos1', 'pa', 'pc', 'py',
       'ints', 'tdp', 'ra', 'sra', 'ry', 'tdr', 'fuml', 'trg', 'rec', 'recy',
       'tdrec', 'exp', 'height', 'weight', 'dob', 'dv', 'forty', 'bench',
       'vertical', 'broad', 'shuttle', 'cone', 'arm', 'hand'],
      dtype='object')

### Cumulative Stats

In [32]:
df.sort_values(by=['player', 'gid'], ascending=True)

,pk,gid,seas,wk,player,team,pos1,pa,pc,py,...,dob,dv,forty,bench,vertical,broad,shuttle,cone,arm,hand
83134,3999&AA-0025,3999,2015,1,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
83361,4010&AA-0025,4010,2015,2,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
83916,4036&AA-0025,4036,2015,3,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
84240,4052&AA-0025,4052,2015,4,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
84447,4062&AA-0025,4062,2015,5,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
84666,4072&AA-0025,4072,2015,6,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
84957,4086&AA-0025,4086,2015,7,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
85180,4096&AA-0025,4096,2015,8,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
85729,4123&AA-0025,4123,2015,10,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
86033,4137&AA-0025,4137,2015,11,AA-0025,DET,RB,0,0,0,...,1993-06-13,Big Ten,4.5,24,42.5,130,3.95,6.79,30.0,8.625
